In [62]:
import json
import requests
import threading
import urllib

In [63]:
inputFile = "../input/sari_abzug-utf-8_23_04-tsv.txt"
manifestDirectory = "../manifests/"
limit = 100

In [64]:
with open(inputFile, 'r') as f:
    rawData = json.load(f)

In [65]:
rowsWithManifests = [d for d in rawData['rows'] if d['manifest'] is not None]

In [66]:
urlsAndFilenames = [{
    "manifest": d['manifest'],
    "filename": manifestDirectory + urllib.parse.quote(d['manifest'], safe='') + '.json'
} for d in rowsWithManifests]

In [67]:
def fetchManuscript(url, filename):
    urlHandler = urllib.request.urlopen(url)
    data = urlHandler.read()
    content = json.loads(data.decode('utf-8'))
    with open(filename, 'w') as f:
        json.dump(content, f, indent=4)

In [68]:
test = urlsAndFilenames[1]
fetchManuscript(test['manifest'], test['filename'])

In [69]:
threads = [threading.Thread(target=fetchManuscript, args=(d['manifest'], d['filename'])) for d in urlsAndFilenames[:limit]]
for thread in threads:
    thread.start()
for thread in threads:
    thread.join()